# Install, Paths and Parameters

In [1]:
import os
from pathlib import Path
import getpass
import numpy as np
import time
import torch
import torchattacks
from torchattacks import *
from tqdm import tqdm
import random
import sys

# allow imports when running script from within project dir
[sys.path.append(i) for i in ['.', '..']]

# garbage collection from previous runs
import gc
gc.collect()
torch.cuda.empty_cache()

# local
from src.helpers.helpers import get_random_indexes, get_random_classes
from src.model.dino import get_dino
from src.model.data import create_loader

# seed
SEED = 42
random.seed(SEED)
gc.collect()
torch.manual_seed(SEED)
np.random.seed(SEED)

In [7]:
username = getpass.getuser()
DATA_PATH = Path('/','cluster', 'scratch', username, 'dl_data')
ORG_LABEL_PATH = Path(DATA_PATH, 'correct_labels.txt')
ORIGINAL_IMAGES_PATH = Path(DATA_PATH,'ori_data','ImageNetClasses')

INDEX_SUBSET = get_random_indexes()

BATCH_SIZE = 1

DEVICE = 'cuda'

In [8]:
#!python ../setup/collect_env.py

# Import DINO
Official repo: https://github.com/facebookresearch/dino

In [9]:
model = get_dino(DEVICE)

Using cache found in /cluster/home/thobauma/.cache/torch/hub/facebookresearch_dino_main


# Load data

In [10]:
org_loader = create_loader(ORIGINAL_IMAGES_PATH, ORG_LABEL_PATH, INDEX_SUBSET, BATCH_SIZE)

# Attack model

In [11]:
attacks = [
    FGSM(model, eps=0.062),
    CW(model, c=50, lr=0.00155, steps=30)
]

In [12]:

for atk in attacks:
    
    print("-"*70)
    print(atk)
    
    correct = 0
    clean_correct = 0
    total = 0
    
    for images, labels, _ in tqdm(org_loader):
        
        start = time.time()

        cuda_images = images.to(DEVICE)
        clean_outputs = model(cuda_images, False)
        del cuda_images

        adv_images = atk(images, labels)
        labels = labels.to(DEVICE)
        outputs = model(adv_images)

        _, pre = torch.max(outputs.data, 1)
        _, pre_clean = torch.max(clean_outputs.data, 1)

        total += len(labels.flatten())
        correct += (pre == labels).sum()
        clean_correct += (pre_clean == labels).sum()

        del images
        del labels
        torch.cuda.empty_cache()

    print('Total elapsed time (sec): %.2f' % (time.time() - start))
    print('Accuracy against attack: %.2f %%' % (100 * float(correct) / total))
    print('Clean accuracy: %.2f %%' % (100 * float(clean_correct) / total))

  0%|          | 0/1000 [00:00<?, ?it/s]

----------------------------------------------------------------------
FGSM(model_name=ViTModel, device=cuda:0, eps=0.062, attack_mode=default, return_type=float)


  0%|          | 0/1000 [00:00<?, ?it/s]

Total elapsed time (sec): 0.10
Accuracy against attack: 1.00 %
Clean accuracy: 78.80 %
----------------------------------------------------------------------
CW(model_name=ViTModel, device=cuda:0, c=50, kappa=0, steps=30, lr=0.00155, attack_mode=default, return_type=float)


100%|██████████| 1000/1000 [08:52<00:00,  1.88it/s]

Total elapsed time (sec): 0.78
Accuracy against attack: 0.00 %
Clean accuracy: 78.80 %


In [6]:
!nvidia-smi

Fri Dec 17 16:59:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:04:00.0 Off |                  N/A |
| 27%   29C    P8     5W / 180W |      2MiB /  8119MiB |      0%   E. Process |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 1080    Off  | 00000000:0D:00.0 Off |                  N/A |
| 27%   

In [5]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   90948 KB |  102956 KB |  102956 KB |   12008 KB |
|       from large pool |   83504 KB |   95504 KB |   95504 KB |   12000 KB |
|       from small pool |    7444 KB |    7452 KB |    7452 KB |       8 KB |
|---------------------------------------------------------------------------|
| Active memory         |   90948 KB |  102956 KB |  102956 KB |   12008 KB |
|       from large pool |   83504 KB |   95504 KB |   95504 KB |